In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# import engarde.decorators as ed

pd.set_option('display.max_colwidth', None)
%config InlineBackend.figure_format = 'retina'


def read(file):
    df = ((pd.read_excel(file, skiprows=2, usecols=columns))
          .rename(columns, axis=1)
          .pipe(str_to_numeric, ['fat', 'carbs', 'fiber', 'protein'])
          .pipe(str_to_lower, ['name', 'category'])
          .pipe(strip_strings, ['name'])
#           .assign(net_carbs = lambda x: x.carbs - x.fiber)
          .dropna(how='any'))
    return df


columns = {
    'Name': 'name',
    'Kategorie': 'category',
    'Bezugseinheit': 'units',
    'Energie, Kalorien': 'calories',
    'Fett, total (g)': 'fat',
    'Kohlenhydrate, verfügbar (g)': 'carbs',
    'Nahrungsfasern (g)': 'fiber',
    'Protein (g)': 'protein'
}


def strip_strings(df, columns):
    """
    Remove leading and trailing whitespace from strings.
    """
    df = df.copy()
    def strip(col):
        return col.str.strip()
    
    df[columns] = df[columns].apply(strip)
    return df
    
    
def str_to_numeric(df, columns):
    """
    Convert string columns to numeric.
    """
    df = df.copy()
    def converter(col):
        return pd.to_numeric(col, errors='coerce')

    df[columns] = df[columns].apply(converter)
    return df


def str_to_lower(df, columns):
    """
    Convert string columns to lower-case.
    """
    df = df.copy()
    def converter(col):
        return col.str.lower()
    
    df[columns] = df[columns].apply(converter)
    return df


def find_foods_item(data, item):
    return data[data.name.str.contains(item)]

def find_myfoods_item(data, item):
    return data.loc[data.index.str.contains(item)]



def make_myfoods(data, foods):
    df = data.copy()
    df = df.loc[data.name.isin(foods)]
    df = df.rename(columns={'name': 'ingredient'})
    df['ingredient'] = df['ingredient'].map(foods)
    
    return df


my_foods = {
    'avocado, roh': 'avocado',
    'blattsalat (durchschnitt), roh': 'lettuce',
    'bier, lager': 'beer',
    'blumenkohl, gedämpft (ohne zugabe von salz)': 'cauliflower',
    'champignon, roh': 'mushrooms',    
    'feige, getrocknet': 'fig',
    'fleisch (durchschnitt exkl. innereien), roh': 'meat',
    'forelle, ganz, gekocht (ohne zusatz von fett und salz)': 'trout',
    'gemüse (durchschnitt), roh': 'vegetables',
    'hart- und halbhartkäse, vollfett (durchschnitt)': 'cheese',
    'haferflocken': 'oats',
    'halbrahm, pasteurisiert': 'half-and-half cream', 
    'halbentrahmte milch 1.5% fett, uht': 'milk',
    'hühnerei, ganz, festgekocht': 'eggs',
    'hüttenkäse, nature': 'cottage cheese',
    'joghurt, mokka': 'mocca yoghurt',
    'joghurt, nature': 'nature yoghurt',
    'kakaopulver': 'cocoa powder', 
    'kartoffel, geschält, roh': 'potatoes',
    'kochbutter': 'butter',
    'kokosfett': 'coconut fat',
    'kokosnuss, getrocknet (kokosrapseln, kokosflocken)': 'coconut flakes',
    'leinöl, kaltgepresst': 'flaxseed oil',
    'mandel': 'almond',
    'olivenöl': 'olive oil',
    'peperoni, rot, roh': 'red pepper',
    'poulet, schenkel, mit haut, gebraten (ohne zusatz von fett und salz)': 'chicken leg',
    'poulet, brust, schnitzel oder geschnetzeltes, gebraten (ohne zusatz von fett und salz)': 'chicken breast',
    'reis poliert, gekocht in salzwasser (unjodiert)': 'rice',
    'samen, kerne, nüsse (durchschnitt), ungesalzen': 'nuts and seed mix',
    'schokolade, dunkel (bitter)': 'dark chocolate',
    'schwein, hals, steak, gebraten (ohne zusatz von fett und salz)': 'pork steak',
    'spinat, roh': 'spinach', 
    'tofu': 'tofu',
    'vollrahm, uht': 'cream'
}


foods = read('Schweizer-Nährwertdatenbank-V6.1.xlsx')
# foods = foods.set_index('name')
myfoods = make_myfoods(foods, my_foods).set_index('ingredient')

%store foods myfoods

Stored 'foods' (DataFrame)
Stored 'myfoods' (DataFrame)


In [2]:
find_foods_item(foods, 'kokos')

,name,category,units,calories,fat,carbs,fiber,protein
428,kokosfett,fette und öle/fette,pro 100g essbarer Anteil,894,99.0,0.0,0.0,0.8
429,kokosmakronen,süssigkeiten/guetzli,pro 100g essbarer Anteil,558,40.1,35.8,12.7,7.1
430,kokosnuss,"nüsse, samen und ölfrüchte",pro 100g essbarer Anteil,479,42.0,16.0,9.5,4.6
431,"kokosnuss, getrocknet (kokosrapseln, kokosflocken)","nüsse, samen und ölfrüchte",pro 100g essbarer Anteil,667,63.3,6.4,20.0,8.1


In [3]:
find_myfoods_item(myfoods, 'pep')

,category,units,calories,fat,carbs,fiber,protein
ingredient,,,,,,,
red pepper,gemüse/gemüse frisch,pro 100g essbarer Anteil,32,0.4,5.2,2.0,0.9


In [6]:
myfoods

,category,units,calories,fat,carbs,fiber,protein
ingredient,,,,,,,
avocado,"gemüse/gemüse frisch;nüsse, samen und ölfrüchte",pro 100g essbarer Anteil,144,14.2,0.8,3.0,1.8
beer,alkoholhaltige getränke/bier,pro 100 ml,42,0.0,3.1,0.0,0.5
lettuce,gemüse/gemüse frisch,pro 100g essbarer Anteil,18,0.3,1.4,1.9,1.6
cauliflower,gemüse/gemüse gekocht (inkl. konserven),pro 100g essbarer Anteil,28,0.3,2.4,2.5,2.5
mushrooms,gemüse/pilze,pro 100g essbarer Anteil,25,0.3,1.1,1.9,3.7
fig,früchte/früchte getrocknet,pro 100g essbarer Anteil,273,1.2,56.8,11.0,3.2
meat,"fleisch und innereien/kalb;fleisch und innereien/geflügel;fleisch und innereien/lamm, schaf;fleisch und innereien/rind;fleisch und innereien/schwein;fleisch und innereien/wild",pro 100g essbarer Anteil,145,6.6,0.0,0.0,21.4
trout,fisch/süsswasserfische,pro 100g essbarer Anteil,159,8.8,0.0,0.0,20.1
vegetables,gemüse/gemüse frisch,pro 100g essbarer Anteil,26,0.3,3.7,2.0,1.3


Find items